In [1]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import math

# Fixed parameters

In [2]:
samplesize = 5000
order = 2

# Variable parameters (qubits and layers)

In [3]:
#List of number of qubits
qubits = [3]

#List of number of layers
layers = [1, 2, 3, 4, 5]

# Multiple layer fidelities 'order' moment calculations

In [4]:
def designtest(order, qubits, samples, circuit, nlayers):
    fidel_moment = 0
    for i in range(samples):
        #angles1 = np.array([np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
        #             requires_grad=True)
        #angles2 = np.array([np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
        #             requires_grad=True)
        
        angles1 = np.array( [ [[np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
                   [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
                   [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
                   [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)]] for j in range(nlayers)],
                           requires_grad = True)
        
        angles2 = np.array( [ [[np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
                   [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
                   [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
                   [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)]] for j in range(nlayers)], 
                           requires_grad = True)
        
        state1 = circuit(angles1)
        state2 = circuit(angles2)
        
        F = np.abs( np.dot(state1, state2) )**2
        
        fidel_moment = fidel_moment + F**order
        
    fidel_moment = fidel_moment/samples
    
    expressib = fidel_moment - (math.factorial(order) * math.factorial(2**qubits-1))/(math.factorial(order
                                                                                                    +2**qubits-1))
    
    return expressib

# Code that executes the different conditions

In [5]:
now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")


for wires in qubits:
    dev = qml.device("default.qubit", wires=wires, shots=None)
        
    
    for nlayers in layers:
        
        #No connections
        def layer_noconnec(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
                qml.RX(rots[2][i], wires=[i])
                qml.RY(rots[3][i], wires=[i])
        
        def noconnec(rots):
            qml.layer(layer_noconnec, nlayers, rots)
            return qml.state()

        noconnec_circuit = qml.QNode(noconnec, dev)
        
        
        
        #Linear
        def layer_linear(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
            for i in range(wires):
                qml.RX(rots[2][i], wires=[i])
                qml.RY(rots[3][i], wires=[i])

        def linear(rotations):
            qml.layer(layer_linear, nlayers, rotations)
            return qml.state()

        linear_circuit = qml.QNode(linear, dev)
        
        
        
        #Ring
        def layer_ring(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
            qml.CNOT(wires=[wires-1,0])
            for i in range(wires):
                qml.RX(rots[2][i], wires=[i])
                qml.RY(rots[3][i], wires=[i])

        def ring(rotations):
            qml.layer(layer_ring, nlayers, rotations)
            return qml.state()


        ring_circuit = qml.QNode(ring, dev)
        
        
        noconnec_design = designtest(order, wires, samplesize, noconnec_circuit, nlayers)
        linear_design = designtest(order, wires, samplesize, linear_circuit, nlayers)
        ring_design = designtest(order, wires, samplesize, ring_circuit, nlayers)
        
        f = open("designtest_ansatz2_{0}design_qubits{1}_layers{2}_samples{3}.txt"
                 .format(order,wires,nlayers, samplesize), "w")
        
        print('Circuit,Designtest', file = f)
        print('No connections,', noconnec_design, file = f)
        print('Linear,', linear_design, file = f)
        print('Ring,', ring_design, file = f)
        
        print('Execution: qubits=',wires,'; layers=', nlayers)
        print()
        
now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")


f.close()
print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Execution: qubits= 3 ; layers= 1

Execution: qubits= 3 ; layers= 2

Execution: qubits= 3 ; layers= 3

Execution: qubits= 3 ; layers= 4

Execution: qubits= 3 ; layers= 5

Tempo inicial: 
2023-09-11 14:28:32
Tempo final:
2023-09-11 15:46:37
